In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../utilities')
from basic_utilities import *
from model_utilities import *
from pandas.plotting import scatter_matrix
from ml_utilities import sk_feature_selection
import scipy.stats as stats
import pdb
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
raw_data = pd.read_csv('../../../data/Base.csv')

In [4]:
raw_data.shape

(1000000, 32)

In [5]:
raw_data.columns.tolist()

['fraud_bool',
 'income',
 'name_email_similarity',
 'prev_address_months_count',
 'current_address_months_count',
 'customer_age',
 'days_since_request',
 'intended_balcon_amount',
 'payment_type',
 'zip_count_4w',
 'velocity_6h',
 'velocity_24h',
 'velocity_4w',
 'bank_branch_count_8w',
 'date_of_birth_distinct_emails_4w',
 'employment_status',
 'credit_risk_score',
 'email_is_free',
 'housing_status',
 'phone_home_valid',
 'phone_mobile_valid',
 'bank_months_count',
 'has_other_cards',
 'proposed_credit_limit',
 'foreign_request',
 'source',
 'session_length_in_minutes',
 'device_os',
 'keep_alive_session',
 'device_distinct_emails_8w',
 'device_fraud_count',
 'month']

In [6]:
numeric_vars, str_vars, oth_vars = sort_vars(raw_data, raw_data.columns.tolist())

In [7]:
numeric_vars

['fraud_bool',
 'income',
 'name_email_similarity',
 'prev_address_months_count',
 'current_address_months_count',
 'customer_age',
 'days_since_request',
 'intended_balcon_amount',
 'zip_count_4w',
 'velocity_6h',
 'velocity_24h',
 'velocity_4w',
 'bank_branch_count_8w',
 'date_of_birth_distinct_emails_4w',
 'credit_risk_score',
 'email_is_free',
 'phone_home_valid',
 'phone_mobile_valid',
 'bank_months_count',
 'has_other_cards',
 'proposed_credit_limit',
 'foreign_request',
 'session_length_in_minutes',
 'keep_alive_session',
 'device_distinct_emails_8w',
 'device_fraud_count',
 'month']

In [8]:
low_variation_vars =  sk_feature_selection.f_low_variation(raw_data, numeric_vars, 0.05)
low_variation_vars

,feature,rescaled_std,recommendation,outliers
0,email_is_free,0.499106,keep,False
1,keep_alive_session,0.494044,keep,False
2,phone_home_valid,0.493076,keep,False
3,has_other_cards,0.416251,keep,False
4,bank_months_count,0.378634,keep,False
5,proposed_credit_limit,0.364712,keep,False
6,income,0.362928,keep,False
7,month,0.315713,keep,False
8,phone_mobile_valid,0.313293,keep,False
9,name_email_similarity,0.290447,keep,False


In [9]:
numeric_vars.remove('device_fraud_count')
numeric_vars

['fraud_bool',
 'income',
 'name_email_similarity',
 'prev_address_months_count',
 'current_address_months_count',
 'customer_age',
 'days_since_request',
 'intended_balcon_amount',
 'zip_count_4w',
 'velocity_6h',
 'velocity_24h',
 'velocity_4w',
 'bank_branch_count_8w',
 'date_of_birth_distinct_emails_4w',
 'credit_risk_score',
 'email_is_free',
 'phone_home_valid',
 'phone_mobile_valid',
 'bank_months_count',
 'has_other_cards',
 'proposed_credit_limit',
 'foreign_request',
 'session_length_in_minutes',
 'keep_alive_session',
 'device_distinct_emails_8w',
 'month']

In [10]:
str_vars

['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']

In [11]:
switch_vars = ['mobile_phone_vali']

In [12]:
oth_vars

[]

In [13]:
w = pd.ExcelWriter('proc_means.xlsx')
with w as writer:
    proc_means(raw_data, numeric_vars).to_excel(writer,sheet_name = 'all', startcol = 2, startrow = 2)
    proc_means(raw_data.loc[raw_data['fraud_bool']==1], numeric_vars).to_excel(writer,sheet_name = 'fraud=1', startcol = 2, startrow = 2)

In [14]:
raw_data['orig_weight'] = 1

In [15]:
summary_by_month = pivot(raw_data, varlist={'fraud_bool':('weighted_avg','orig_weight'),'income':'mean', 
                                            'credit_risk_score':'mean'}, by_vars = ['month'])
summary_by_month

,month,count,WA_fraud_bool,mean_income,mean_credit_risk_score
0,0,132440,0.011326,0.5,117.0
1,1,127620,0.009387,0.5,109.0
2,2,136979,0.008746,0.5,111.0
3,3,150936,0.009222,0.6,146.0
4,4,127691,0.011371,0.6,137.0
5,5,119323,0.011825,0.6,142.0
6,6,108168,0.013405,0.6,148.0
7,7,96843,0.014746,0.6,142.0


In [16]:
qcut_vars = ['income','name_email_similarity', 'prev_address_months_count', 'current_address_months_count', 'customer_age', \
            'intended_balcon_amount',  'velocity_6h', 'velocity_24h', 'velocity_4w', 'bank_branch_count_8w', \
            'date_of_birth_distinct_emails_4w', 'credit_risk_score','bank_months_count','proposed_credit_limit', \
            'session_length_in_minutes','device_distinct_emails_8w']

In [17]:
qcut_dict = {}
for var in qcut_vars:
    qcut_dict[var] = [i*0.05 for i in range(1,20)]

In [18]:
raw_data, cps = binning_q(raw_data, qcut_dict)

In [19]:
raw_data.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,velocity_6h_bin,velocity_24h_bin,velocity_4w_bin,bank_branch_count_8w_bin,date_of_birth_distinct_emails_4w_bin,credit_risk_score_bin,bank_months_count_bin,proposed_credit_limit_bin,session_length_in_minutes_bin,device_distinct_emails_8w_bin
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,nan,nan,nan,"(2.0, 5.0]","(4.0, 5.0]","(152.0, 165.0]","(5.0, 10.0]","(1000.0, 1500.0]","(15.6, 21.7]",nan
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,"(8782.0, 9639.0]","(5518.0, 5753.0]","(5677.0, 6002.0]","(2.0, 5.0]","(16.0, 19.0]","(152.0, 165.0]","(1.0, 2.0]","(1000.0, 1500.0]","(3.1, 3.5]",nan
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,"(4206.0, 4559.0]","(5298.0, 5518.0]","(5677.0, 6002.0]","(13.0, 15.0]","(10.0, 11.0]","(83.0, 92.0]","(29.0, 30.0]","(199.0, 500.0]",nan,nan
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,nan,"(6368.0, 6821.0]","(5677.0, 6002.0]","(9.0, 11.0]","(12.0, 13.0]","(83.0, 92.0]","(-1.1, 1.0]","(199.0, 500.0]","(12.4, 15.6]",nan
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,"(7147.0, 7681.0]","(5102.0, 5298.0]","(5677.0, 6002.0]","(-0.1, 1.0]","(5.0, 6.0]","(83.0, 92.0]","(25.0, 26.0]","(199.0, 500.0]","(3.5, 3.9]",nan


In [20]:
figs = []
for var in qcut_vars:
    var_bin = f"{var}_bin"
    xy = pivot(raw_data, varlist={'fraud_bool':'logodds', var:'mean'}, by_vars = [var_bin])
    figs.append(px_scatter_plot(xy, f'mean_{var}', 'logodds_fraud_bool', show=False))

In [21]:
if os.path.exists("logodds_plots.html"):
  os.remove("logodds_plots.html")
    
with open('logodds_plots.html', 'a') as file:
    for f in figs:
        file.write(f.to_html())

In [22]:
raw_data['zip_count_4w_level'] = raw_data['zip_count_4w'].apply(lambda x: 'low' if x <= 950 else ('mid' if 950 < x < 1850 else 'high'))

In [23]:
raw_data['zip_count_4w_level'].value_counts()

zip_count_4w_level
mid     438266
low     288674
high    273060
Name: count, dtype: int64

In [24]:
raw_data['month'].value_counts()

month
3    150936
2    136979
0    132440
4    127691
1    127620
5    119323
6    108168
7     96843
Name: count, dtype: int64

In [25]:
figs = []
for var in qcut_vars:
    var_bin = f"{var}_bin"

    xy1 = pivot(raw_data.loc[raw_data['zip_count_4w_level']=='low'], varlist={'fraud_bool':'logodds', var:'mean'}, by_vars = ['month',var_bin])
    xy2 = pivot(raw_data.loc[raw_data['zip_count_4w_level']=='mid'], varlist={'fraud_bool':'logodds', var:'mean'}, by_vars = ['month',var_bin])
    xy3 = pivot(raw_data.loc[raw_data['zip_count_4w_level']=='high'], varlist={'fraud_bool':'logodds', var:'mean'}, by_vars = ['month',var_bin])

    xy1['zip_count_4w_level'] = 'low'
    xy2['zip_count_4w_level'] = 'mid'
    xy3['zip_count_4w_level'] = 'high'
    
    xy = pd.concat([xy1,xy2,xy3],axis=0)

    figs.append(px_scatter_plot(xy, f'mean_{var}', 'logodds_fraud_bool', by_var1 = 'month', by_var2 = 'zip_count_4w_level' ,show=False))


In [26]:
if os.path.exists("logodds_plots_level.html"):
  os.remove("logodds_plots_level.html")

In [27]:
with open('logodds_plots_level.html', 'a') as file:
    for f in figs:
        file.write(f.to_html())

In [28]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','employment_status')

In [29]:
col_pct

employment_status,CA,CB,CC,CD,CE,CF,CG
fraud_bool,,,,,,,
0,0.729398,0.138867,0.037237,0.026717,0.022892,0.044439,0.000451
1,0.806873,0.086409,0.084504,0.009067,0.004806,0.007707,0.000635


In [30]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','phone_home_valid')
col_pct

phone_home_valid,0,1
fraud_bool,,
0,0.581094,0.418906
1,0.746940,0.253060


In [31]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','phone_mobile_valid')
col_pct

phone_mobile_valid,0,1
fraud_bool,,
0,0.109888,0.890112
1,0.149424,0.850576


In [32]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','has_other_cards')
col_pct

has_other_cards,0,1
fraud_bool,,
0,0.775467,0.224533
1,0.915586,0.084414


In [33]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','foreign_request')
col_pct

foreign_request,0,1
fraud_bool,,
0,0.975038,0.024962
1,0.949678,0.050322


In [34]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','source')
col_pct

source,INTERNET,TELEAPP
fraud_bool,,
0,0.992987,0.007013
1,0.989845,0.010155


In [35]:
freq, row_pct, col_pct = cross_tab(raw_data,'fraud_bool','keep_alive_session')
col_pct

keep_alive_session,0,1
fraud_bool,,
0,0.420429,0.579571
1,0.658355,0.341645


In [36]:
ttest = grp_ttest(qcut_vars,raw_data.loc[raw_data['fraud_bool']==0], raw_data.loc[raw_data['fraud_bool']==1])

In [37]:
ttest

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,intended_balcon_amount,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,bank_months_count,proposed_credit_limit,session_length_in_minutes,device_distinct_emails_8w
T-statistic,-45.125003,36.744379,26.039489,-33.72034,-63.083737,24.531801,16.894752,11.183663,11.536384,11.577269,43.264096,-70.800347,3.222285,-69.070733,-8.999825,-35.726857
P-value,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001272,0.0,0.0,0.0


In [38]:
selector = sk_feature_selection()

In [39]:
cont_vars = ['income','name_email_similarity', 'prev_address_months_count', 'current_address_months_count', 'customer_age', \
            'intended_balcon_amount', 'zip_count_4w', 'velocity_6h', 'velocity_24h', 'velocity_4w', 'bank_branch_count_8w', \
            'date_of_birth_distinct_emails_4w', 'credit_risk_score','bank_months_count','proposed_credit_limit', \
            'session_length_in_minutes','device_distinct_emails_8w','month']

In [40]:
disc_vars = ['email_is_free','phone_home_valid','phone_mobile_valid','has_other_cards','foreign_request','keep_alive_session']

In [41]:
mi = selector.f_normalized_mi_matrix(raw_data, ['fraud_bool']+cont_vars+disc_vars)

In [42]:
mi.to_excel('nmi_matrix.xlsx')

In [43]:
coef = selector.f_feature_select(raw_data, cont_vars + disc_vars, 'fraud_bool', mtype='classification', chart='off')

In [44]:
coef.to_excel('feature_importance.xlsx')